In [17]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from _plotly_future_ import v4_subplots
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.plotly as py
import plotly.graph_objs as go
init_notebook_mode(connected=True)
import pickle 

In [28]:
# parameter settings
S_min   = 50          # lower bound for the asset price
S_max   = 150         # upper bound for the asset price
Tau_min = 0.01        # lower bound for time to maturity
Tau_max = 1           # upper bound for time to maturity
K       = 100         # strike price
r       = 0.0         # risk-free interest rate                  
sig     = 0.25        # volatility               
d       = 0.0         # dividend rate                
steps   = 60          # steps 

In [29]:
def calculate_delta(S,K,r,d,Tau,sig):
    Z = (np.log(S/K) + (r-d+sig**2/2) * Tau) / (sig*np.sqrt(Tau))
    return  np.exp(-d*Tau) *norm.cdf(Z)

In [30]:
# Delta surface
Delta = np.zeros([steps, steps])

Tau_grid = np.arange(Tau_min, Tau_max+0.01, (Tau_max-Tau_min)/(steps-1))
S_grid = np.arange(S_min, S_max+0.1, (S_max-S_min)/(steps-1))

for i, S in enumerate(S_grid):
    for j, Tau in enumerate(Tau_grid):
        Delta[i][j] = calculate_delta(S, K, r, d, Tau, sig) 

In [38]:
layout = go.Layout(title='Delta as function of the time to maturity and the asset price', 
                   scene = dict(xaxis_title = 'Tau', yaxis_title = 'S', zaxis_title = 'Delta', 
                                camera_eye = dict(x=-1.6, y=-1.6, z=0.7)))

fig = go.Figure(data=[go.Surface(z=Delta, x=Tau_grid, y=S_grid)], layout=layout)

iplot(fig)

plot(fig, filename='Delta.html') 

'Delta.html'

In [32]:
def calculate_gamma(S,K,r,d,Tau,sig):
    Z = (np.log(S/K) + (r-d+sig**2/2) * Tau) / (sig*np.sqrt(Tau))
    return np.exp(-d*Tau) * norm.pdf(Z)/(S*sig*np.sqrt(Tau))

def calculate_charm(S,K,r,d,Tau,sig):
    Z = (np.log(S/K) + (r-d+sig**2/2) * Tau) / (sig*np.sqrt(Tau))
    d2 = Z - sig*np.sqrt(Tau)
    A = d*np.exp(-d*Tau) * norm.cdf(Z)
    B = np.exp(-d*Tau) * norm.pdf(Z)* ((2*(r-d)*Tau) - d2*sig*np.sqrt(Tau))/(S*Tau*sig*np.sqrt(Tau))
    return A-B

def calculate_speed(S,K,r,d,Tau,sig):
    gamma = calculate_gamma(S,K,r,d,Tau,sig)
    Z = (np.log(S/K) + (r-d+sig**2/2) * Tau) / (sig*np.sqrt(Tau))
    return -(gamma/S) * (Z/(sig*np.sqrt(Tau))+1)

In [33]:
# Gamma surface
Gamma = np.zeros([steps, steps])

for i, S in enumerate(S_grid):
    for j, Tau in enumerate(Tau_grid):
        Gamma[i][j] = calculate_gamma(S, K, r, d, Tau, sig) 
        
        
layout = go.Layout(title='Gamma as function of the time to maturity and the asset price', 
                   scene = dict(xaxis_title = 'Tau', yaxis_title = 'S', zaxis_title = 'Gamma', 
                                camera_eye = dict(x=-1.6, y=-1.6, z=0.7)))

fig = go.Figure(data=[go.Surface(z=Gamma, x=Tau_grid, y=S_grid)],
                layout=layout)

iplot(fig)

# plot(fig, filename='Gamma.html') 

In [37]:
# Charm surface
Charm = np.zeros([steps, steps])

for i, S in enumerate(S_grid):
    for j, Tau in enumerate(Tau_grid):
        Charm[i][j] = calculate_charm(S, K, r, d, Tau, sig) 
        
        
layout = go.Layout(title='Charm as function of the time to maturity and the asset price', 
                   scene = dict(xaxis_title = 'Tau', yaxis_title = 'S', zaxis_title = 'Charm', 
                                camera_eye = dict(x=-1.6, y=-1.6, z=0.7)))

fig = go.Figure(data=[go.Surface(z=Charm, x=Tau_grid, y=S_grid)],
                layout=layout)


iplot(fig)

plot(fig, filename='Charm.html') 

'Charm.html'

In [12]:
# Speed surface
Speed = np.zeros([steps, steps])

for i, S in enumerate(S_grid):
    for j, Tau in enumerate(Tau_grid):
        Speed[i][j] = calculate_speed(S, K, r, d, Tau, sig) 
        
        
layout = go.Layout(title='Speed as function of the time to maturity and the asset price', 
                   scene = dict(xaxis_title = 'Tau', yaxis_title = 'S', zaxis_title = 'Speed', 
                                camera_eye = dict(x=-1.6, y=-1.6, z=0.7)))

fig = go.Figure(data=[go.Surface(z=Speed, x=Tau_grid, y=S_grid)],
                layout=layout)

iplot(fig)

# plot(fig, filename='Gamma.html') 

In [25]:
np.exp(1)

2.718281828459045